In [2]:
import numpy as np
import pandas as pd

# For pre-processing data 
from sklearn import preprocessing as pp 
from sklearn.compose import ColumnTransformer 


In [7]:
trainData = pd.read_csv("../train_data.csv")
testData = pd.read_csv("../test_data.csv")

